In [541]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import math
import json

import os
import numpy as np
import pandas as pd

from itertools import islice
from datetime import datetime as dt
import datetime

In [927]:
quebec_path = "../data/quebec/Quebec_.csv"
headers = ["Region",  "Total Number of Cases", "Date of Infection", "Lat", "Long"]
df = pd.read_csv(quebec_path)
regions = df[:19]
dates = list(regions)[1:]
dates = [d[:-2] for d in dates]


lat_long = [(48.333333, -68.666667), (49.8667, -71.75), (47.333333, -71.5), (47, -73), (45.481556, -71.667917),
               (45.5, -73.566667), (45.783333, -75.083333), (48.233333, -79.016667), (50.866667, -65.816667),
               (56.166667, -74.416667), (48.683333, -65.4), (46.483333, -70.616667), (45.583333, -73.75),
               (46.766667, -73.833333), (46.433333, -74.983333), (45.383333, -73.1), (46.283333, -72.066667)]


population = [197385, 276368, 729997, 266112, 319004, 1942044, 382604, 146717, 92518, 44561, 90311, 420082, 422993, 494796, 589400, 1507070, 242399]

density = [8.9, 2.8, 38.8, 7.4, 31.2, 3889.8, 12.4, 2.5, 0.4, 0.1, 4.4, 27.9, 1710.9, 39.8, 28.4, 135.4, 35.0]
ICU_CAP = 1000

In [543]:

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def nday_avg(data, n):
    avg = []
    for ndays in window(data, n):
        avg.append(sum(ndays)/n)
    return avg

In [544]:
def per100k(data, population):
    pass

In [928]:
stacked_df = pd.DataFrame(columns=headers)
for ix,row in regions.iterrows():
    region = [list(row)[0]]*(len(row)-1)
    if ix < 17:
        lat = [lat_long[ix][0]]*(len(row)-1)
        long = [lat_long[ix][1]]*(len(row)-1)
    else : 
        lat = ["na"]*(len(row)-1)
        long = ["na"]*(len(row)-1)
    num_cases = list(row)[1:]
    region_list = [region, num_cases, dates, lat, long]
    temp = pd.DataFrame(list(map(list, zip(*region_list))), columns = headers)
    stacked_df =stacked_df.append(temp,  ignore_index=True)


# Total Cases

In [929]:
total_cases = list(df.iloc[19])[1:]

In [930]:
fig = px.bar(stacked_df, x="Date of Infection", y="Total Number of Cases", color = "Region", color_discrete_sequence= px.colors.qualitative.Light24)
fig.add_trace(go.Scatter(x=dates, y=total_cases,
                    mode='lines',
                    name='Total'))


fig.update_layout(legend_orientation="h",  xaxis = {"dtick" : 10})
fig.update_layout(legend=dict( y=-0.45))

fig.show()
with open('../plotly/total_cases.json', 'w') as f:
    f.write(fig.to_json())

In [931]:
all_regions = stacked_df.Region.unique()
regions = [stacked_df.loc[stacked_df['Region'] == r]["Total Number of Cases"].to_list() for r in all_regions]


fig = make_subplots(rows=9, cols=2, subplot_titles=tuple(all_regions[:17]), shared_xaxes=True, shared_yaxes = True)

for  i,m in enumerate(regions[:17]) :

    r = i//2+1
    c = i%2+1

    fig.add_trace(
    go.Scatter(x=dates, y=m, mode = "lines"),
    row=r, col=c
)
    
fig.update_layout(showlegend=False, title_text="Infection over time by Region", height=1000)
fig.show()

with open('../plotly/infections_by_region.json', 'w') as f:
    f.write(fig.to_json())

# Hospitalization

In [932]:
ICU_cap = 1000
hospitalization = list(df.loc[[28]].values)[0][31:]
ICU = list(df.loc[[29]].values)[0][31:]

In [933]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x= dates[30:], y=hospitalization, name="Hospitalized"),

)

fig.add_trace(
    go.Scatter(x=dates[30:], y=ICU, name="ICU"),
)


fig.add_trace(
    go.Scatter(x=dates[30:], y=[ICU_cap]*len(dates[30:]), name="ICU Capacity" , mode='lines',),
)



fig.update_layout(title='Hospitalization',
                   xaxis_title='Date',
                   yaxis_title='Number of Patients')

fig.show()

with open('../plotly/hospitilization.json', 'w') as f:
    f.write(fig.to_json())

# New Cases

In [934]:
new_cases = list(df.iloc[20])[2:]
fiveday_avg = nday_avg(new_cases, 5)

In [935]:
fig = go.Figure(data=go.Scatter(x=dates[1:], y=new_cases,
                    mode='lines',
                    name='New Cases'))

fig.add_trace(go.Scatter(x=dates[8:], y=fiveday_avg,
                    mode='lines',
                    name='Five Day Average', visible = False))


fig.update_layout(
    updatemenus=[
        dict(
            #type="buttons",
            direction="down",
            showactive = True,
            #active = 0,
            x=1,
            y=1.2,
            buttons=list([
                dict(label="New Cases",
                     method="update",
                     args=[{"visible": [True,False ]},
                           {"title": 'New Cases', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Number of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),
                dict(label="5 day average",
                     method="update",
                     args=[{"visible": [False,True]},
                           {"title": 'Average Cases in Last 5 days', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Number of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),

            ]),
        )
    ])

fig.update_layout(title='New Cases Each Day',
                   xaxis_title='Date',
                   yaxis_title='Number of New Cases',  xaxis = {"dtick" : 10})
fig.show()
with open('../plotly/new_cases.json', 'w') as f:
    f.write(fig.to_json())

# Doubling

In [706]:
doubling = list(df.iloc[26])[1:]


In [333]:
fig = go.Figure(data=go.Scatter(x=dates[1:], y=doubling,
                    mode='lines',
                    name='Time to Double'))
fig.update_layout(title='Current Doubling Time (Higher Doubling Time is Better)',
                   xaxis_title='Date',
                   yaxis_title='Days to Double',  xaxis = {"dtick" : 10})
fig.show()
with open('../plotly/doubling.json', 'w') as f:
    f.write(fig.to_json())

# Testing

In [936]:
headers = ["Results",  "People Tested", "Lab Report Date"]
positive = list(df.iloc[20])[3:]
negative = list(df.iloc[22])[3:]

In [937]:
total_negative = list(np.cumsum(negative))
total_positive = list(np.cumsum(positive))
total_tests = total_negative+total_positive
positive_per_test = [ pos/total for pos, total in zip(total_positive, total_tests)]

/opt/anaconda3/envs/plakk/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning:

divide by zero encountered in double_scalars



In [938]:
fig = go.Figure(data=[
    go.Bar(name='Negative', x=dates[5:], y=negative[2:] ,visible = False),
    go.Bar(name='Positive', x=dates[5:], y=positive[2:],  visible = False)])

fig.update_layout(barmode='stack')

fig.add_trace(go.Scatter(x=dates[5:], y=positive_per_test[3:],
                    mode='lines',
                    name='Positive Tests per Total Tests (Cumulative Average)'))

fig.add_trace(go.Scatter(x=dates[5:], y=total_tests[3:],
                    mode='lines',
                    name='Total Tests',  visible = False))

fig.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Positive Tests per Total Test',  xaxis = {"dtick" : 10})

fig.update_layout(
    updatemenus=[
        dict(
            #type="buttons",
            direction="down",
            showactive = True,
            #active = 0,
            x=1,
            y=1.2,
            buttons=list([
                dict(label="Positive Tests Per",
                     method="update",
                     args=[{"visible": [False,False, True, False]},
                           {"title": 'Positive Tests per Total Tests (Cumulative Average)', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Positive Tests per Total Test',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),
                dict(label="Test",
                     method="update",
                     args=[{"visible": [True,True, False, False]},
                           {"title": 'Total Tests', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Number of Tests',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),

            ]),
        )
    ])

fig.show()
with open('../plotly/tested.json', 'w') as f:
    f.write(fig.to_json())

In [939]:
total = list(df.iloc[19][1:])
dead = list(df.iloc[24])[1:]
recovered = list(df.iloc[25])[1:]
active = [ t-r-d for t,r,d in zip(total, recovered, dead)]

dead_percent = [d/t*100 for d,t in zip(dead, total)]
recovered_percent = [r/t*100 for r,t in zip(recovered, total)]
active_percent = [a/t*100 for a,t in zip(active, total)]

In [940]:

fig = go.Figure()


fig.add_trace(
    go.Scatter(x= dates, y=dead, name="Deceased", visible = True),

)

fig.add_trace(
    go.Scatter(x=dates, y=dead_percent, name="Deceased Percent", visible = False),
)


fig.add_trace(
    go.Scatter(x= dates, y=recovered, name="Recovered", visible = True),
)

fig.add_trace(
    go.Scatter(x=dates, y=recovered_percent, name="Recovered Percent", visible = False),
)


fig.add_trace(
    go.Scatter(x= dates, y=active, name="Active"),
)

fig.add_trace(
    go.Scatter(x=dates, y=active_percent, name="Active Percent", visible = False),
)





fig.update_layout(
    updatemenus=[
        dict(
            #type="buttons",
            direction="down",
            showactive = True,
            #active = 0,
            x=1,
            y=1.5,
            buttons=list([
                dict(label="Current Active",
                     method="update",
                     args=[{"visible": [True,False,True, False, True, False]},
                           {"title": 'Active, Deceased and Recovered Cases', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Number of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),
                dict(label="Percentage Active",
                     method="update",
                     args=[{"visible": [False,True,False, True, False, True]},
                           {"title": 'Percentage of Active, Deceased and Recovered Cases', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Percentage of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),

            ]),
        )
    ])


fig.update_layout(
    title_text="Active Cases",
)
fig.update_layout(legend_orientation="h")
fig.update_layout(legend=dict( y=-0.4),  xaxis = {"dtick" : 10})


fig.show()

with open('../plotly/cases.json', 'w') as f:
    f.write(fig.to_json())

# # Add figure title
# fig.update_layout(
#     title_text="Types of cases"
# )

# # Set x-axis title
# fig.update_xaxes(title_text="Date")

# # Set y-axes titles
# fig.update_yaxes(title_text="<b>primary</b> Amount", secondary_y=False)
# fig.update_yaxes(title_text="<b>secondary</b> Percent", secondary_y=True)

# fig.show()

# Maps

In [558]:
with open("../data/quebec/quebec.geojson","r") as infile:
    quebec = json.load(infile)

In [941]:
lastday_df = stacked_df.loc[stacked_df['Date of Infection'] == dates[-1]][:17]
lastday_df = lastday_df.rename(columns = {"Region":"res_nm_reg"})
total_cases = list(lastday_df["Total Number of Cases"])
cases_per_density = [ total/d for total, d in zip(total_cases, density)]
cases_per_population = [ total/p*100000 for total, p in zip(total_cases, population)]
lastday_df["Cases per Density"] = cases_per_density
lastday_df["Cases per 100k"] = cases_per_population

#del lastday_df['Date of Infection']

In [942]:
fig = px.choropleth(lastday_df, geojson=quebec, color="Total Number of Cases", locations="res_nm_reg", 
                    featureidkey= "properties.res_nm_reg", color_continuous_scale='Blues',
                    projection='conic equidistant'
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title='Total Cases by Region')
fig.show()
with open('../plotly/map.json', 'w') as f:
    f.write(fig.to_json())

In [ ]:
fig = px.choropleth(lastday_df, geojson=quebec, color="Cases per 100k", locations="res_nm_reg", 
                    featureidkey= "properties.res_nm_reg", color_continuous_scale='Blues',
                    projection='conic equidistant'
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title='Total Cases per 100k')
fig.show()
with open('../plotly/map_population.json', 'w') as f:
    f.write(fig.to_json())

# Exponential

In [895]:
total = list(df.iloc[19][1:])


region_cases = []

for i in range(17):
    temp = list(df.iloc[i][1:])
    ix_0 = [i for i, t in enumerate(temp) if t >0]
    if len(ix_0) >0 : 
        temp = temp[ix_0[0]:]
    else : 
        temp = []
    region_cases.append(temp)

In [896]:
region_cases_per_100k = []

for i, r in enumerate(region_cases): 
    if r : 
        region_cases_per_100k.append( [ 100000*x/population[i] for x in r ])
    else:
        region_cases_per_100k.append([])

total_cases_per_100k = [ 100000*x/8164361 for x in total ]

regions = stacked_df.Region.unique()


In [897]:
past_week = []
for i,_ in enumerate(total[7:]):
    past_week.append(total[i:i+7]) 
past_week_sum = [100000*(w[-1]-w[0])/8164361 for w in past_week]

region_week = []
for i, r in enumerate(region_cases): 
    if len(r)>=7 :
        temp = []
        for r0,r7 in zip(r,r[7:]):
            temp.append(100000*(r7-r0)/population[i])
        region_week.append( temp)
    else:
        region_week.append([])   


In [898]:
fig = go.Figure()

color = ['#EB89B5', '#5b7bd6','#ff6960', '#d11411', '#00b159', '#00aedb', '#f37735', '#ffc425',
          '#a200ff', '#f47835', '#d41243', '#8ec127', '#feda75', '#fa7e1e', '#962fbf', '#fa9e1e', 
         '#800000', '#FFFF00', '#7D3C98']

fig.add_trace(go.Scatter(y=total_cases_per_100k,name = "All Quebec", mode = 'lines+markers'))

for r_c, r in zip(region_cases_per_100k, regions):
    fig.add_trace(go.Scatter(y=r_c, name = r, mode = 'lines+markers'))


fig.add_trace(go.Scatter(x=total_cases_per_100k[7:], y=past_week_sum,name = "All Quebec", mode = 'lines+markers', visible=False,))


for r_100, r_c, r in zip(region_cases_per_100k, region_week, regions):
    fig.add_trace(go.Scatter(x =r_100[7:] , y=r_c, name = r, mode = 'lines+markers', visible=False))

fig.update_layout(
    xaxis_title_text='Date since 1st infection', # xaxis label
    yaxis_title_text='Cases per 100k (log scale)', # yaxis label

)




fig.update_layout(
    updatemenus=[
        dict(
            #type="buttons",
            direction="down",
            showactive = True,
            #active = 0,
            x=1,
            y=1.5,
            buttons=list([
                dict(label="Growth per Date",
                     method="update",
                     args=[{"visible": [True]*18+ [False]*18},
                           {"title": 'Exponential Growth since First Infection', 
                            "xaxis.title" : 'Days since first infection',
                            "yaxis.title" : 'Cases per 100k (log scale)',
                            "yaxis.type": "linear",
                            "xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),
                dict(label="Growth per total Cases",
                     method="update",
                     args=[{"visible":  [False]*18+ [True]*18},
                           {"title": 'Exponential Growth per total cases',
                            "xaxis.title" : 'Cases per 100k ',
                            "yaxis.title" : 'New Cases per 100k in last week',
                            "yaxis.type": "linear",
                            "xaxis.type": "linear",
                           }]),
                                dict(label="Growth per Date Log Scale",
                     method="update",
                     args=[{"visible": [True]*18+ [False]*18},
                           {"title": 'Exponential Growth since First Infection', 
                            "xaxis.title" : 'Days since first infection',
                            "yaxis.title" : 'Cases per 100k (log scale)',
                            "yaxis.type": "log",
                            "xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),
                dict(label="Growth per total Cases Log Scale",
                     method="update",
                     args=[{"visible":  [False]*18+ [True]*18},
                           {"title": 'Exponential Growth per total cases',
                            "xaxis.title" : 'Cases per 100k (log scale)',
                            "yaxis.title" : 'New Cases per 100k in last week(log scale)',
                            "yaxis.type": "log",
                            "xaxis.type": "log",
                           }]),
            ]),
        )
    ])


fig.update_layout(
    title_text="Exponential",
)
fig.update_layout(legend_orientation="h")
fig.update_layout(legend=dict( y=-0.4))


fig.show()

with open('../plotly/exponential.json', 'w') as f:
    f.write(fig.to_json())

In [566]:
montreal_regions = ['04-Mauricie','14- Lanaudiere', '06-Montreal', '13- Laval']
non_montreal_regions = [r for r in regions if r not in montreal_regions]

In [626]:
montreal_region_pop = 0
non_montreal_region_pop = 0 
for r, p in zip(regions, population):
    if r in montreal_regions:
        montreal_region_pop+= p
    else : 
        non_montreal_region_pop +=p



In [899]:
montreal_region_cases = []
for m in montreal_regions:
    montreal_region_cases.append(df.loc[df['Unnamed: 0'] == m].values[0][1:])

montreal_region_cases = np.sum(np.array(montreal_region_cases), axis = 0 )

non_montreal_region_cases = []
for m in non_montreal_regions:
    non_montreal_region_cases.append(df.loc[df['Unnamed: 0'] == m].values[0][1:])
non_montreal_region_cases = np.sum(np.array(non_montreal_region_cases), axis = 0 )

montreal_region_cases_per_100k = 100000*np.array(montreal_region_cases)/ montreal_region_pop
non_montreal_region_cases_per_100k = 100000*np.array(non_montreal_region_cases)/ non_montreal_region_pop

In [900]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=montreal_region_cases,name = "Montreal and Surrounding Regions", mode = 'lines+markers'))
fig.add_trace(go.Scatter(y=non_montreal_region_cases,name = "Rest of Quebec", mode = 'lines+markers'))

fig.show()

In [901]:
new_cases_montreal_region =[]

for m0, m1 in zip(montreal_region_cases[:-1], montreal_region_cases[1:]):
    new_cases_montreal_region.append(m1-m0)
    
new_cases_non_montreal_region =[]

for m0, m1 in zip(non_montreal_region_cases[:-1], non_montreal_region_cases[1:]):
    new_cases_non_montreal_region.append(m1-m0)
    
fiveday_avg_montreal = nday_avg(new_cases_montreal_region, 5)
fiveday_avg_non_montreal = nday_avg(new_cases_non_montreal_region, 5)

In [902]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=new_cases_montreal_region, x=dates[1:], name = "Montreal and Surrounding Regions", mode = 'lines'))
fig.add_trace(go.Scatter(y=new_cases_non_montreal_region, x=dates[1:],name = "Rest of Quebec", mode = 'lines'))

fig.add_trace(go.Scatter(y=fiveday_avg_montreal, x=dates[6:], name = "Montreal and Surrounding Regions", mode = 'lines', visible = False))
fig.add_trace(go.Scatter(y=fiveday_avg_non_montreal, x=dates[6:],name = "Rest of Quebec", mode = 'lines', visible = False))



fig.update_layout(
    updatemenus=[
        dict(
            #type="buttons",
            direction="down",
            showactive = True,
            #active = 0,
            x=1,
            y=1.2,
            buttons=list([
                dict(label="New Cases",
                     method="update",
                     args=[{"visible": [True,True, False,False ]},
                           {"title": 'New Cases', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Number of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),
                dict(label="5 day average",
                     method="update",
                     args=[{"visible": [False, False, True,True]},
                           {"title": 'Average Cases in Last 5 days', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Number of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),

            ]),
        )
    ])

fig.update_layout(title='New Cases Montreal and Neighbouring Regions vs Rest',
                   xaxis_title='Date',
                   yaxis_title='Number of New Cases',  xaxis = {"dtick" : 10})
fig.show()
with open('../plotly/new_cases_regions.json', 'w') as f:
    f.write(fig.to_json())

# Montreal

In [37]:
with open("../data/quebec/montreal.geojson","r") as infile:
    montreal_nhood_map = json.load(infile)
    

In [38]:
with open("../data/quebec/montreal_ciuss.geojson","r") as infile:
    ciuss_map = json.load(infile)

In [903]:
data_path = '../data/quebec/'
paths = os.listdir(data_path)

In [910]:
path = [p for p in paths if 'ciuss2020' in p][0] 
with open(os.path.join(data_path, path), 'r') as outfile:
    montreal = json.load( outfile)

In [911]:
path = [p for p in paths if 'nhood2020' in p][0] 
with open(os.path.join(data_path, path), 'r') as outfile:
    montreal_nhood = json.load( outfile)

In [906]:
data_past_path = '../data/quebec/past'
past_paths = os.listdir(data_past_path)
past_ciuss_paths = [(p, dt.strptime(p[14:33], "%Y-%m-%dT%H:%M:%S")) for p in past_paths if 'ciuss' in p]
o = []
for p, date in sorted(past_ciuss_paths):
    f = open(os.path.join(data_past_path, p))
    m = json.load(f)
    for k in m:
        if k.startswith('CIUSSS'):
            val = m[k][0]
            if val == ' TBC':
                continue
        
            o.append([k.strip(), date, int(val.replace(",","").replace(" ","").replace(".",""))])
df = pd.DataFrame(o, columns=['CIUSS', 'Date', 'Cases'])
fig = px.line(df, x='Date', y='Cases', color='CIUSS', title='Cases per CIUSS over time')
fig.show()

with open('../plotly/montreal_deaths.json', 'w') as f:
    f.write(fig.to_json())

In [862]:
nhood_map = {
    'Côte-des-Neiges–Notre-Dame-de-Grâce' : 'Côte-des-Neiges-Notre-Dame-de-Grâce',
    'Plateau Mont-Royal' : 'Le Plateau-Mont-Royal',
    "Baie D'urfé" : "Baie-d'Urfé",
    'Mercier–Hochelaga-Maisonneuve' : 'Mercier-Hochelaga-Maisonneuve',
    'Côte-Saint-Luc' : 'Côte-Saint-Luc',
    'Kirkland' : 'Kirkland',
    "L'Île-Bizard–Sainte-Geneviève" :"L'Île-Bizard-Sainte-Geneviève",
    'Pierrefonds–Roxboro' :'Pierrefonds-Roxboro',
    'Rivière-des-Prairies–Pointe-aux-Trembles' : 'Rivière-des-Prairies-Pointe-aux-Trembles',
    'Rosemont–La Petite Patrie' : 'Rosemont-La Petite-Patrie',
    'Saint-Léonard' : 'Saint-Léonard',
    'Senneville' : 'Senneville',
    'Ahuntsic–Cartierville' :'Ahuntsic-Cartierville',
    'Sud-Ouest' : 'Le Sud-Ouest',
    'Villeray–Saint-Michel–Parc-Extension' :'Villeray-Saint-Michel-Parc-Extension',
    'Montréal Est': 'Montréal-Est',
    'Rosemont-La Petite Patrie': 'Rosemont-La Petite-Patrie'
    
}

In [912]:
keys = list(montreal.keys())

montreal_map = {}
# montreal_rates = {}

for key,val in montreal.items() :
    montreal_map[key.strip()] = int(val[0].replace(",","").replace(" ","").replace(".",""))
    #montreal_rates[key] = float(val[1].replace(",",""))

# del montreal_map['Total  Montréal']
# del montreal_map['Territoire à confirmer²']

if 'Total for Montréal' in montreal_map:
    del montreal_map['Total for Montréal']
if 'Territory to be confirmed²' in montreal_map:
    del montreal_map['Territory to be confirmed²']
if 'Territoire à confirmer' in montreal_map:
    del montreal_map['Territoire à confirmer']
if 'Total à Montréal' in montreal_map:
    del montreal_map['Total à Montréal']

In [913]:
keys = list(montreal_nhood.keys())

montreal_map_nhood = {}
# montreal_rates = {}

for key,val in montreal_nhood.items() :
    stripped = key.strip()
    stripped = stripped.replace('\x96', '-')

    if stripped in nhood_map : 
        montreal_map_nhood[nhood_map[stripped]] = int(val[0].replace(",","").replace("< ","").replace("<","").replace(" ",""))
    else : 
        montreal_map_nhood[stripped] = int(val[0].replace(",","").replace("< ","").replace("<","").replace(" ",""))
# del montreal_map_nhood['Total à Montréal']
# del montreal_map_nhood['Territoire à confirmer²']
if 'Total for Montréal' in montreal_map_nhood:
    del montreal_map_nhood['Total for Montréal']
if 'Territory to be confirmed²' in montreal_map_nhood:
    del montreal_map_nhood['Territory to be confirmed²']
if 'Territoire à confirmer' in montreal_map_nhood:
    del montreal_map_nhood['Territoire à confirmer']
if 'Total à Montréal' in montreal_map_nhood:
    del montreal_map_nhood['Total à Montréal']
    #montreal_rates[key] = float(val[1].replace(",",""))

In [914]:
montreal_df = pd.DataFrame(montreal_map.items(), columns=['CIUSS', 'Cases'])

In [915]:
fig = px.choropleth(montreal_df, geojson=ciuss_map, color="Cases", locations="CIUSS", 
                    featureidkey= "properties.NOM",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title='Cases per CIUSS')
fig.show()
with open('../plotly/map_montreal.json', 'w') as f:
    f.write(fig.to_json())

In [916]:
montreal_df = pd.DataFrame(montreal_map_nhood.items(), columns=['Neighbourhoods', 'Cases'])

In [917]:
fig = px.choropleth(montreal_df, geojson=montreal_nhood_map, color="Cases", locations='Neighbourhoods', 
                    featureidkey= "properties.NOM",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title='Cases per Neighbourhood')
fig.show()
with open('../plotly/map_montreal_nhood.json', 'w') as f:
    f.write(fig.to_json())

In [918]:
path = [p for p in paths if 'age_death' in p][0] 
with open(os.path.join(data_path, path), 'r') as outfile:
    age_deaths = json.load( outfile)

In [919]:
path = [p for p in paths if 'age_cases' in p][0] 
with open(os.path.join(data_path, path), 'r') as outfile:
    age_case = json.load( outfile)

In [920]:
del age_deaths["Âge à déterminer"]

In [921]:
age_deaths_map = {}
for age, deaths in age_deaths.items():
    if age == 'Moins de 20 ans':
        key = '<20'
    elif age == '90 ans et plus': 
        key = '>90'
    else :
        key = age.strip(" ans")
    age_deaths_map[key] = float(deaths.replace(",","."))

In [922]:
deaths_ages_df = pd.DataFrame(age_deaths_map.items(), columns=['Ages', 'Deaths (%)'])

In [923]:
del age_case['Âge à déterminer']

In [924]:
age_cases_map = {}
for age, deaths in age_case.items():
    if age == '90 ans ou plus': 
        key = '>90'
    else :
        key = age.strip(" ans")
    age_cases_map[key] = float(deaths.replace(",","."))

In [925]:
ages_df = pd.DataFrame(age_cases_map.items(), columns=['Ages', 'Cases (%)'])

In [926]:
fig = go.Figure([go.Bar(x=deaths_ages_df["Ages"], y=deaths_ages_df['Deaths (%)'])])

fig.add_trace(
    go.Bar(x=ages_df["Ages"], y=ages_df['Cases (%)'], visible = False),
)

fig.update_layout(
    xaxis_title_text='Age ranges', # xaxis label
    yaxis_title_text='Percentage', # yaxis label

)

fig.update_layout(
    updatemenus=[
        dict(
            #type="buttons",
            direction="down",
            showactive = True,
            #active = 0,
            x=1,
            y=1.5,
            buttons=list([
                dict(label="Deaths Age",
                     method="update",
                     args=[{"visible": [True,False]},
                           {"title": 'Deaths by Age', 
                            "xaxis.title" : 'Age',
                            "yaxis.title" : 'Percentage',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),
                dict(label="Cases Age",
                     method="update",
                     args=[{"visible": [False,True]},
                           {"title": 'Cases by Age', 
                            "xaxis.title" : 'Ages',
                            "yaxis.title" : 'Percentage of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),

            ]),
        )
    ])


fig.update_layout(
    title_text="Deaths by Age",
)
fig.update_layout(legend_orientation="h")
fig.update_layout(legend=dict( y=-0.4))

fig.show()

with open('../plotly/age.json', 'w') as f:
    f.write(fig.to_json())